# Exercise 3.1  Pose compositions 

## Theoretical background

A fundamental aspect of the development of mobile robots is the motion itself. This is not a trivial matter as it is one of the main sources of uncertainty and other constraints to the movement difficult its implementation.
This particular lesson introduces the concept of a robot's pose and how we deal with it in a probabilistic context.

The pose itself can take multiple forms depending on the problems context:

- **2D location**: In a planar context we only need to a 2d vector $[x, y]^T$ to locate a robot against a point of reference, the origin $(0, 0).$
- **2D pose**: In most cases involving mobile robots, the location alone is insufficient. We need an aditional paramater known as orientation or *bearing*. Therefore, a robot's pose is usually expressed as $[x, y, \theta]^T$. *In the rest of the course, we mostly refer to this one.*
- **3D pose**: Although we will only mentioned in passing, for robotics applications in the 3D space, *i.e.* UAV or drones, not only a third axis $z$ is added, but to handle the orientation in a 3D environment we need 3 components, *i.e.* roll, pitch and yaw. This course is centered around planar movile robots so we will not use this one, nevertheless most methods could be adapted to 3D environments.

In [ ]:
%matplotlib notebook

# IMPORTS

import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

from utils.DrawRobot import DrawRobot
from utils.tcomp import tcomp

## 3.1.1

**Context**

In this exercise we will handle the robot's movement as a **composition of poses**.

Given an initial pose $p_1$ and a pose differential $\Delta p$, *i.e.* how much the robot has moved during an interval of time, we compute the final pose $p$ using the composition function:

$$
    p_1 = 
        \begin{bmatrix}
            x_1 \\ y_1 \\ \theta_1
        \end{bmatrix},
    \Delta p = 
        \begin{bmatrix}
            \Delta x \\ \Delta y \\ \Delta \theta
        \end{bmatrix}
$$

$$
    \begin{equation}
    p = \begin{bmatrix}
            x \\ y \\ \theta
        \end{bmatrix}
        = p_1 \oplus \Delta p
        = \begin{bmatrix}
            x_1 + \Delta x \cos \theta_1 - \Delta y \sin \theta_1 \\ 
            y_1 + \Delta x \sin \theta_1 - \Delta y \cos \theta_1 \\
            \theta_1 + \Delta \theta
          \end{bmatrix}
    \end{equation}
$$


The differential $\Delta p$, although we are using it as control in this exercise, normally is calculated given the robot's locomotion or sensed by the wheel encoders.

**Assignment**

Modify the main function in the next cell for the robot to describe a $8m \times 8m$ square path as seen in the figure below.

The robot starts in the bottom-left corner $(0, 0)$ and heads north in increments of $2 m$. Each 4 steps it will turn right.

**Example**


<figure style="text-align:center">
  <img src="images/fig3-1-1.png" alt="">
  <figcaption>Fig. 1: Route of our robot.</figcaption>
</figure>

In [ ]:
class Robot():
    '''Mobile robot implementation
    
        Attr:
            pose: Expected position of the robot
    '''
    def __init__(self, mean):
        self.pose = mean

    def step(self, u):
        self.pose = tcomp(self.pose, u)
    
    def draw(self, fig, ax):
        DrawRobot(fig, ax, self.pose)

def main(robot):
    
    # PARAMETERS
    num_steps = 4
    u = np.vstack([2., 0., 0.])
    angle_inc = -np.pi/2
    
    # MATPLOTLIB
    fig, ax = plt.subplots()
    plt.ion()
    plt.draw()
    plt.xlim((-2, 10))
    plt.ylim((-2, 10))
    
    plt.grid()
    robot.draw(fig, ax)
    
    # TODO Change main loop to move in a square
    for _ in range(num_steps):
        robot.step(u)
        
        robot.draw(fig, ax)
        fig.canvas.draw()
        plt.pause(0.1)
        
# RUN

initial_pose = np.vstack([0., 0., np.pi/2])
robot = Robot(initial_pose)
main(robot)

## 3.1.2

**Context**

In the previous case, the robot motion was error-free, this is overly optimistic as in a real use case the conditions of the environment are a huge source of uncertainty.

Therefore, we have to transform the movement of the robot into a (multidimensional) gaussian distribution.

- The mean is still the pose differential in the previous exercise.
- The covariance is a $3 \times 3$ matrix, which defines the amount of error at each step (time interval) 

**Assignment**

Now, add a Gaussian noise to the motion. Along with the expected pose, draw the real pose of the robot(affected by noise), assuming the incremental motion is:

$$
    \Delta p = N(\Delta p_{given}, \Sigma_{\Delta p})
    \textit{ with } 
    \Sigma_{\Delta p}  =
        \begin{bmatrix}
            0.04 & 0 & 0 \\
            0 & 0.04 & 0 \\
            0 & 0 & 0.01
        \end{bmatrix}
    (\text{ units in }m^2 \text{ and } rad^2)
$$

Run the cell several times to see that the motion (and the path) is different each time. Try also with different values of the covariance matrix.

Complete this new class that extends our previous Robot by adding some amount of noise to the movement.

**Example**

<figure style="text-align:center">
  <img src="images/fig3-1-2.png" alt="">
  <figcaption>Fig. 2: Movement of our robot using pose compositions. <br/>
      Containing the expected poses (in red) and the true pose <br/> affected by noise (in blue)</figcaption>
</figure>

In [ ]:
class NoisyRobot(Robot):
    """Mobile robot implementation. It's motion has a set ammount of noise.
    
        Attr:
            pose: Inherited from Robot
            true_pose: Real robot pose, which has been affected by some ammount of noise.
            covariance: Amount of error of each step.
    """
    def __init__(self, mean, covariance):
        super().__init__(mean)
        self.true_pose = mean
        self.covariance = covariance
        
    def step(self, step_increment):
        """Computes a single step of our noisy robot.
        
            super().step(...) updates the expected pose (without noise)
            Generate a noisy increment based on step_increment and self.covariance.
            Then this noisy increment is applied to self.true_pose
        """
        super().step(step_increment)
        # TODO: Implement noisy movement
        raise NotImplementedError
        
    def draw(self, fig, ax):
        super().draw(fig, ax)
        DrawRobot(fig, ax, self.true_pose, color='blue')

# RUN

initial_pose = np.vstack([0., 0., np.pi/2])
cov = np.diag([0.0, 0.0, 0.012])  

robot = NoisyRobot(initial_pose, cov)
main(robot)